In [1]:
#importing the essential libraries
import pandas as pd
import numpy as np
import ast
from sklearn.preprocessing import MultiLabelBinarizer, LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer

from nltk.stem.porter import PorterStemmer


In [3]:
#reading the dataset
dataframe1 = pd.read_csv("tmdb_5000_credits.csv")
dataframe2 = pd.read_csv("tmdb_5000_movies.csv").drop(['homepage', 'popularity', 'budget', 'revenue', 'status', 'original_title', 'popularity', 'runtime', 'production_countries', 'production_companies', 'spoken_languages', 'vote_average', 'vote_count', 'id', 'tagline', 'original_language'], axis = 1)
df1 = dataframe1.copy()
df2 = dataframe2.copy()

In [5]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   genres        4803 non-null   object
 1   keywords      4803 non-null   object
 2   overview      4800 non-null   object
 3   release_date  4802 non-null   object
 4   title         4803 non-null   object
dtypes: object(5)
memory usage: 187.7+ KB


In [7]:
df2

,genres,keywords,overview,release_date,title
0,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","In the 22nd century, a paraplegic Marine is di...",2009-12-10,Avatar
1,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...","[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...","Captain Barbossa, long believed to be dead, ha...",2007-05-19,Pirates of the Caribbean: At World's End
2,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",A cryptic message from Bond’s past sends him o...,2015-10-26,Spectre
3,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...","[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",Following the death of District Attorney Harve...,2012-07-16,The Dark Knight Rises
4,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 818, ""name"": ""based on novel""}, {""id"":...","John Carter is a war-weary, former military ca...",2012-03-07,John Carter
...,...,...,...,...,...
4798,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...","[{""id"": 5616, ""name"": ""united states\u2013mexi...",El Mariachi just wants to play his guitar and ...,1992-09-04,El Mariachi
4799,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 10749, ""...",[],A newlywed couple's honeymoon is upended by th...,2011-12-26,Newlyweds
4800,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 18, ""nam...","[{""id"": 248, ""name"": ""date""}, {""id"": 699, ""nam...","""Signed, Sealed, Delivered"" introduces a dedic...",2013-10-13,"Signed, Sealed, Delivered"
4801,[],[],When ambitious New York attorney Sam is sent t...,2012-05-03,Shanghai Calling


In [9]:
df1.head()

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [11]:
#preprocessing the dataset
df2.isnull().sum()

genres          0
keywords        0
overview        3
release_date    1
title           0
dtype: int64

In [13]:

df3 = df2.dropna(subset = ['overview', 'release_date'])

df3.isnull().sum()

genres          0
keywords        0
overview        0
release_date    0
title           0
dtype: int64

In [15]:
df3

,genres,keywords,overview,release_date,title
0,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","In the 22nd century, a paraplegic Marine is di...",2009-12-10,Avatar
1,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...","[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...","Captain Barbossa, long believed to be dead, ha...",2007-05-19,Pirates of the Caribbean: At World's End
2,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",A cryptic message from Bond’s past sends him o...,2015-10-26,Spectre
3,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...","[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",Following the death of District Attorney Harve...,2012-07-16,The Dark Knight Rises
4,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 818, ""name"": ""based on novel""}, {""id"":...","John Carter is a war-weary, former military ca...",2012-03-07,John Carter
...,...,...,...,...,...
4798,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...","[{""id"": 5616, ""name"": ""united states\u2013mexi...",El Mariachi just wants to play his guitar and ...,1992-09-04,El Mariachi
4799,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 10749, ""...",[],A newlywed couple's honeymoon is upended by th...,2011-12-26,Newlyweds
4800,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 18, ""nam...","[{""id"": 248, ""name"": ""date""}, {""id"": 699, ""nam...","""Signed, Sealed, Delivered"" introduces a dedic...",2013-10-13,"Signed, Sealed, Delivered"
4801,[],[],When ambitious New York attorney Sam is sent t...,2012-05-03,Shanghai Calling


In [17]:
df3['genres'].loc[1]
 


'[{"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 28, "name": "Action"}]'

In [19]:
df4 = df3.copy()
df4['genres'] = df4['genres'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
df4['genres'] = df4['genres'].apply(lambda x: [genre["name"] for genre in x] if isinstance(x, list) else [])





In [21]:
df4['genres']

0       [Action, Adventure, Fantasy, Science Fiction]
1                        [Adventure, Fantasy, Action]
2                          [Action, Adventure, Crime]
3                    [Action, Crime, Drama, Thriller]
4                [Action, Adventure, Science Fiction]
                            ...                      
4798                        [Action, Crime, Thriller]
4799                                [Comedy, Romance]
4800               [Comedy, Drama, Romance, TV Movie]
4801                                               []
4802                                    [Documentary]
Name: genres, Length: 4799, dtype: object

In [23]:
df5 = df4.copy()
mlb = MultiLabelBinarizer()

encoded_genres = pd.DataFrame(mlb.fit_transform(df5['genres']), columns = mlb.classes_)
encoded_genres.insert(0, 'movie', df5['title']) 

In [25]:
encoded_genres

,movie,Action,Adventure,Animation,Comedy,Crime,Documentary,Drama,Family,Fantasy,...,History,Horror,Music,Mystery,Romance,Science Fiction,TV Movie,Thriller,War,Western
0,Avatar,1,1,0,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0
1,Pirates of the Caribbean: At World's End,1,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,Spectre,1,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,The Dark Knight Rises,1,0,0,0,1,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
4,John Carter,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4794,Sanctuary: Quite a Conundrum,1,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4795,Bang,0,0,0,1,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
4796,Primer,0,0,0,1,0,0,1,0,0,...,0,0,0,0,1,0,1,0,0,0
4797,Cavite,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [27]:
df5

,genres,keywords,overview,release_date,title
0,"[Action, Adventure, Fantasy, Science Fiction]","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","In the 22nd century, a paraplegic Marine is di...",2009-12-10,Avatar
1,"[Adventure, Fantasy, Action]","[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...","Captain Barbossa, long believed to be dead, ha...",2007-05-19,Pirates of the Caribbean: At World's End
2,"[Action, Adventure, Crime]","[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",A cryptic message from Bond’s past sends him o...,2015-10-26,Spectre
3,"[Action, Crime, Drama, Thriller]","[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",Following the death of District Attorney Harve...,2012-07-16,The Dark Knight Rises
4,"[Action, Adventure, Science Fiction]","[{""id"": 818, ""name"": ""based on novel""}, {""id"":...","John Carter is a war-weary, former military ca...",2012-03-07,John Carter
...,...,...,...,...,...
4798,"[Action, Crime, Thriller]","[{""id"": 5616, ""name"": ""united states\u2013mexi...",El Mariachi just wants to play his guitar and ...,1992-09-04,El Mariachi
4799,"[Comedy, Romance]",[],A newlywed couple's honeymoon is upended by th...,2011-12-26,Newlyweds
4800,"[Comedy, Drama, Romance, TV Movie]","[{""id"": 248, ""name"": ""date""}, {""id"": 699, ""nam...","""Signed, Sealed, Delivered"" introduces a dedic...",2013-10-13,"Signed, Sealed, Delivered"
4801,[],[],When ambitious New York attorney Sam is sent t...,2012-05-03,Shanghai Calling


In [29]:
df6 = df5.copy()
df6['keywords'] = df6['keywords'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
df6['keywords'] = df6['keywords'].apply(lambda x: [keyword["name"] for keyword in x] if isinstance(x, list) else [])


In [31]:
df6['release_date'] = pd.to_datetime(df6['release_date'], errors='coerce').dt.year
df6

,genres,keywords,overview,release_date,title
0,"[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","In the 22nd century, a paraplegic Marine is di...",2009,Avatar
1,"[Adventure, Fantasy, Action]","[ocean, drug abuse, exotic island, east india ...","Captain Barbossa, long believed to be dead, ha...",2007,Pirates of the Caribbean: At World's End
2,"[Action, Adventure, Crime]","[spy, based on novel, secret agent, sequel, mi...",A cryptic message from Bond’s past sends him o...,2015,Spectre
3,"[Action, Crime, Drama, Thriller]","[dc comics, crime fighter, terrorist, secret i...",Following the death of District Attorney Harve...,2012,The Dark Knight Rises
4,"[Action, Adventure, Science Fiction]","[based on novel, mars, medallion, space travel...","John Carter is a war-weary, former military ca...",2012,John Carter
...,...,...,...,...,...
4798,"[Action, Crime, Thriller]","[united states–mexico barrier, legs, arms, pap...",El Mariachi just wants to play his guitar and ...,1992,El Mariachi
4799,"[Comedy, Romance]",[],A newlywed couple's honeymoon is upended by th...,2011,Newlyweds
4800,"[Comedy, Drama, Romance, TV Movie]","[date, love at first sight, narration, investi...","""Signed, Sealed, Delivered"" introduces a dedic...",2013,"Signed, Sealed, Delivered"
4801,[],[],When ambitious New York attorney Sam is sent t...,2012,Shanghai Calling


In [33]:
#merging the dataset
df7 = df6.merge(df1)
df7['keywords'] = df7['keywords'].apply(lambda x: x if x else None)
df7['genres'] = df7['genres'].apply(lambda x: x if x else None)
df7['keywords'] = df7['keywords'].fillna('Unknown')
df7['genres'] = df7['genres'].fillna('Unknown')

In [35]:
df7['crew'][3]

'[{"credit_id": "52fe4781c3a36847f81398c3", "department": "Sound", "gender": 2, "id": 947, "job": "Original Music Composer", "name": "Hans Zimmer"}, {"credit_id": "52fe4781c3a36847f8139899", "department": "Production", "gender": 0, "id": 282, "job": "Producer", "name": "Charles Roven"}, {"credit_id": "52fe4781c3a36847f813991b", "department": "Writing", "gender": 2, "id": 525, "job": "Screenplay", "name": "Christopher Nolan"}, {"credit_id": "52fe4781c3a36847f8139865", "department": "Directing", "gender": 2, "id": 525, "job": "Director", "name": "Christopher Nolan"}, {"credit_id": "52fe4781c3a36847f8139893", "department": "Production", "gender": 2, "id": 525, "job": "Producer", "name": "Christopher Nolan"}, {"credit_id": "52fe4781c3a36847f8139915", "department": "Writing", "gender": 2, "id": 525, "job": "Story", "name": "Christopher Nolan"}, {"credit_id": "52fe4781c3a36847f813990f", "department": "Writing", "gender": 2, "id": 527, "job": "Screenplay", "name": "Jonathan Nolan"}, {"credit_

In [37]:
df8 = df7.copy()
df8['cast'] = df8['cast'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
df8['cast'] = df8['cast'].apply(lambda x: [keyword["name"] for keyword in x[:4]] if isinstance(x, list) else [])
df8['crew'] = df8['crew'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
df8['crew'] = df8['crew'].apply(lambda x: [person['name'] for person in x if person['job'] == 'Director'] if isinstance(x, list) else [])


In [39]:
df8['cast'][3]

['Christian Bale', 'Michael Caine', 'Gary Oldman', 'Anne Hathaway']

In [41]:
df8

,genres,keywords,overview,release_date,title,movie_id,cast,crew
0,"[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","In the 22nd century, a paraplegic Marine is di...",2009,Avatar,19995,"[Sam Worthington, Zoe Saldana, Sigourney Weave...",[James Cameron]
1,"[Adventure, Fantasy, Action]","[ocean, drug abuse, exotic island, east india ...","Captain Barbossa, long believed to be dead, ha...",2007,Pirates of the Caribbean: At World's End,285,"[Johnny Depp, Orlando Bloom, Keira Knightley, ...",[Gore Verbinski]
2,"[Action, Adventure, Crime]","[spy, based on novel, secret agent, sequel, mi...",A cryptic message from Bond’s past sends him o...,2015,Spectre,206647,"[Daniel Craig, Christoph Waltz, Léa Seydoux, R...",[Sam Mendes]
3,"[Action, Crime, Drama, Thriller]","[dc comics, crime fighter, terrorist, secret i...",Following the death of District Attorney Harve...,2012,The Dark Knight Rises,49026,"[Christian Bale, Michael Caine, Gary Oldman, A...",[Christopher Nolan]
4,"[Action, Adventure, Science Fiction]","[based on novel, mars, medallion, space travel...","John Carter is a war-weary, former military ca...",2012,John Carter,49529,"[Taylor Kitsch, Lynn Collins, Samantha Morton,...",[Andrew Stanton]
...,...,...,...,...,...,...,...,...
4800,"[Action, Crime, Thriller]","[united states–mexico barrier, legs, arms, pap...",El Mariachi just wants to play his guitar and ...,1992,El Mariachi,9367,"[Carlos Gallardo, Jaime de Hoyos, Peter Marqua...",[Robert Rodriguez]
4801,"[Comedy, Romance]",Unknown,A newlywed couple's honeymoon is upended by th...,2011,Newlyweds,72766,"[Edward Burns, Kerry Bishé, Marsha Dietlein, C...",[Edward Burns]
4802,"[Comedy, Drama, Romance, TV Movie]","[date, love at first sight, narration, investi...","""Signed, Sealed, Delivered"" introduces a dedic...",2013,"Signed, Sealed, Delivered",231617,"[Eric Mabius, Kristin Booth, Crystal Lowe, Geo...",[Scott Smith]
4803,Unknown,Unknown,When ambitious New York attorney Sam is sent t...,2012,Shanghai Calling,126186,"[Daniel Henney, Eliza Coupe, Bill Paxton, Alan...",[Daniel Hsia]


In [43]:
df8['overview'] = df8['overview'].apply(lambda x: x.split())
df8.isnull().sum()

genres          0
keywords        0
overview        0
release_date    0
title           0
movie_id        0
cast            0
crew            0
dtype: int64

In [45]:
df8['genres'] = df8['genres'].apply(lambda x: [i.replace(' ', '') for i in x])
df8['keywords'] = df8['keywords'].apply(lambda x: [i.replace(' ', '') for i in x])
df8['cast'] = df8['cast'].apply(lambda x: [i.replace(' ', '') for i in x])
df8['crew'] = df8['crew'].apply(lambda x: [i.replace(' ', '') for i in x])
df8

,genres,keywords,overview,release_date,title,movie_id,cast,crew
0,"[Action, Adventure, Fantasy, ScienceFiction]","[cultureclash, future, spacewar, spacecolony, ...","[In, the, 22nd, century,, a, paraplegic, Marin...",2009,Avatar,19995,"[SamWorthington, ZoeSaldana, SigourneyWeaver, ...",[JamesCameron]
1,"[Adventure, Fantasy, Action]","[ocean, drugabuse, exoticisland, eastindiatrad...","[Captain, Barbossa,, long, believed, to, be, d...",2007,Pirates of the Caribbean: At World's End,285,"[JohnnyDepp, OrlandoBloom, KeiraKnightley, Ste...",[GoreVerbinski]
2,"[Action, Adventure, Crime]","[spy, basedonnovel, secretagent, sequel, mi6, ...","[A, cryptic, message, from, Bond’s, past, send...",2015,Spectre,206647,"[DanielCraig, ChristophWaltz, LéaSeydoux, Ralp...",[SamMendes]
3,"[Action, Crime, Drama, Thriller]","[dccomics, crimefighter, terrorist, secretiden...","[Following, the, death, of, District, Attorney...",2012,The Dark Knight Rises,49026,"[ChristianBale, MichaelCaine, GaryOldman, Anne...",[ChristopherNolan]
4,"[Action, Adventure, ScienceFiction]","[basedonnovel, mars, medallion, spacetravel, p...","[John, Carter, is, a, war-weary,, former, mili...",2012,John Carter,49529,"[TaylorKitsch, LynnCollins, SamanthaMorton, Wi...",[AndrewStanton]
...,...,...,...,...,...,...,...,...
4800,"[Action, Crime, Thriller]","[unitedstates–mexicobarrier, legs, arms, paper...","[El, Mariachi, just, wants, to, play, his, gui...",1992,El Mariachi,9367,"[CarlosGallardo, JaimedeHoyos, PeterMarquardt,...",[RobertRodriguez]
4801,"[Comedy, Romance]","[U, n, k, n, o, w, n]","[A, newlywed, couple's, honeymoon, is, upended...",2011,Newlyweds,72766,"[EdwardBurns, KerryBishé, MarshaDietlein, Cait...",[EdwardBurns]
4802,"[Comedy, Drama, Romance, TVMovie]","[date, loveatfirstsight, narration, investigat...","[""Signed,, Sealed,, Delivered"", introduces, a,...",2013,"Signed, Sealed, Delivered",231617,"[EricMabius, KristinBooth, CrystalLowe, GeoffG...",[ScottSmith]
4803,"[U, n, k, n, o, w, n]","[U, n, k, n, o, w, n]","[When, ambitious, New, York, attorney, Sam, is...",2012,Shanghai Calling,126186,"[DanielHenney, ElizaCoupe, BillPaxton, AlanRuck]",[DanielHsia]


In [47]:
df8['tags'] = df8['overview'] + df8['genres'] + df8['keywords'] + df8['cast'] + df8['crew']


In [49]:
df9 = df8.drop(['genres', 'keywords', 'overview', 'cast', 'crew', 'release_date'], axis = 1)

df9['title'] = df9['title'].str.lower()
df9

,title,movie_id,tags
0,avatar,19995,"[In, the, 22nd, century,, a, paraplegic, Marin..."
1,pirates of the caribbean: at world's end,285,"[Captain, Barbossa,, long, believed, to, be, d..."
2,spectre,206647,"[A, cryptic, message, from, Bond’s, past, send..."
3,the dark knight rises,49026,"[Following, the, death, of, District, Attorney..."
4,john carter,49529,"[John, Carter, is, a, war-weary,, former, mili..."
...,...,...,...
4800,el mariachi,9367,"[El, Mariachi, just, wants, to, play, his, gui..."
4801,newlyweds,72766,"[A, newlywed, couple's, honeymoon, is, upended..."
4802,"signed, sealed, delivered",231617,"[""Signed,, Sealed,, Delivered"", introduces, a,..."
4803,shanghai calling,126186,"[When, ambitious, New, York, attorney, Sam, is..."


In [51]:
col = df9.pop('movie_id')
df9.insert(0, 'movie_id', col)
col2 = df9.pop('title')
df9.insert(1, 'title', col2)
df9

,movie_id,title,tags
0,19995,avatar,"[In, the, 22nd, century,, a, paraplegic, Marin..."
1,285,pirates of the caribbean: at world's end,"[Captain, Barbossa,, long, believed, to, be, d..."
2,206647,spectre,"[A, cryptic, message, from, Bond’s, past, send..."
3,49026,the dark knight rises,"[Following, the, death, of, District, Attorney..."
4,49529,john carter,"[John, Carter, is, a, war-weary,, former, mili..."
...,...,...,...
4800,9367,el mariachi,"[El, Mariachi, just, wants, to, play, his, gui..."
4801,72766,newlyweds,"[A, newlywed, couple's, honeymoon, is, upended..."
4802,231617,"signed, sealed, delivered","[""Signed,, Sealed,, Delivered"", introduces, a,..."
4803,126186,shanghai calling,"[When, ambitious, New, York, attorney, Sam, is..."


In [53]:
#apply stemming
ps = PorterStemmer()
def stem(text):
    y = " ".join([ps.stem(i) for i in text])
    return y
    

In [55]:
df9['tags'] = df9['tags'].apply(stem)
df9['tags']

0       in the 22nd century, a parapleg marin is dispa...
1       captain barbossa, long believ to be dead, ha c...
2       a cryptic messag from bond’ past send him on a...
3       follow the death of district attorney harvey d...
4       john carter is a war-weary, former militari ca...
                              ...                        
4800    el mariachi just want to play hi guitar and ca...
4801    a newlyw couple' honeymoon is upend by the arr...
4802    "signed, sealed, delivered" introduc a dedic q...
4803    when ambiti new york attorney sam is sent to s...
4804    ever sinc the second grade when he first saw h...
Name: tags, Length: 4805, dtype: object

In [57]:
#applying the vectorization
vector = CountVectorizer(stop_words = 'english', lowercase = True, max_features = 5000)
matrix = vector.fit_transform(df9['tags']).toarray()
print(matrix.shape)

(4805, 5000)


In [59]:
matrix

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [61]:
vector.get_feature_names_out()

array(['000', '007', '10', ..., 'zone', 'zoo', 'zooeydeschanel'],
      dtype=object)

In [63]:
#applying the cosine similarity
from sklearn.metrics.pairwise import cosine_similarity

similarity = cosine_similarity(matrix)

In [65]:
#recommending the movie
def recommend(movie):
    index = df9[df9['title']==movie].index[0]
    distances = similarity[index]
    movies = sorted(list(enumerate( distances)), reverse = True, key = lambda x: x[1])[1:5]

    for i in movies:
        print(df9['title'][i[0]])

In [71]:

movie = str(input('Enter the movie: '))

recommend(movie.lower())

Enter the movie:  batman


batman
batman & robin
batman begins
batman returns


In [67]:
df10 = df8.drop(['tags'], axis = 1)
df10['title'] = df10['title'].str.lower()
df10

,genres,keywords,overview,release_date,title,movie_id,cast,crew
0,"[Action, Adventure, Fantasy, ScienceFiction]","[cultureclash, future, spacewar, spacecolony, ...","[In, the, 22nd, century,, a, paraplegic, Marin...",2009,avatar,19995,"[SamWorthington, ZoeSaldana, SigourneyWeaver, ...",[JamesCameron]
1,"[Adventure, Fantasy, Action]","[ocean, drugabuse, exoticisland, eastindiatrad...","[Captain, Barbossa,, long, believed, to, be, d...",2007,pirates of the caribbean: at world's end,285,"[JohnnyDepp, OrlandoBloom, KeiraKnightley, Ste...",[GoreVerbinski]
2,"[Action, Adventure, Crime]","[spy, basedonnovel, secretagent, sequel, mi6, ...","[A, cryptic, message, from, Bond’s, past, send...",2015,spectre,206647,"[DanielCraig, ChristophWaltz, LéaSeydoux, Ralp...",[SamMendes]
3,"[Action, Crime, Drama, Thriller]","[dccomics, crimefighter, terrorist, secretiden...","[Following, the, death, of, District, Attorney...",2012,the dark knight rises,49026,"[ChristianBale, MichaelCaine, GaryOldman, Anne...",[ChristopherNolan]
4,"[Action, Adventure, ScienceFiction]","[basedonnovel, mars, medallion, spacetravel, p...","[John, Carter, is, a, war-weary,, former, mili...",2012,john carter,49529,"[TaylorKitsch, LynnCollins, SamanthaMorton, Wi...",[AndrewStanton]
...,...,...,...,...,...,...,...,...
4800,"[Action, Crime, Thriller]","[unitedstates–mexicobarrier, legs, arms, paper...","[El, Mariachi, just, wants, to, play, his, gui...",1992,el mariachi,9367,"[CarlosGallardo, JaimedeHoyos, PeterMarquardt,...",[RobertRodriguez]
4801,"[Comedy, Romance]","[U, n, k, n, o, w, n]","[A, newlywed, couple's, honeymoon, is, upended...",2011,newlyweds,72766,"[EdwardBurns, KerryBishé, MarshaDietlein, Cait...",[EdwardBurns]
4802,"[Comedy, Drama, Romance, TVMovie]","[date, loveatfirstsight, narration, investigat...","[""Signed,, Sealed,, Delivered"", introduces, a,...",2013,"signed, sealed, delivered",231617,"[EricMabius, KristinBooth, CrystalLowe, GeoffG...",[ScottSmith]
4803,"[U, n, k, n, o, w, n]","[U, n, k, n, o, w, n]","[When, ambitious, New, York, attorney, Sam, is...",2012,shanghai calling,126186,"[DanielHenney, ElizaCoupe, BillPaxton, AlanRuck]",[DanielHsia]


In [69]:
df10 = df10[['movie_id', 'title', 'genres', 'keywords', 'overview', 'release_date', 'cast', 'crew']]
df10

,movie_id,title,genres,keywords,overview,release_date,cast,crew
0,19995,avatar,"[Action, Adventure, Fantasy, ScienceFiction]","[cultureclash, future, spacewar, spacecolony, ...","[In, the, 22nd, century,, a, paraplegic, Marin...",2009,"[SamWorthington, ZoeSaldana, SigourneyWeaver, ...",[JamesCameron]
1,285,pirates of the caribbean: at world's end,"[Adventure, Fantasy, Action]","[ocean, drugabuse, exoticisland, eastindiatrad...","[Captain, Barbossa,, long, believed, to, be, d...",2007,"[JohnnyDepp, OrlandoBloom, KeiraKnightley, Ste...",[GoreVerbinski]
2,206647,spectre,"[Action, Adventure, Crime]","[spy, basedonnovel, secretagent, sequel, mi6, ...","[A, cryptic, message, from, Bond’s, past, send...",2015,"[DanielCraig, ChristophWaltz, LéaSeydoux, Ralp...",[SamMendes]
3,49026,the dark knight rises,"[Action, Crime, Drama, Thriller]","[dccomics, crimefighter, terrorist, secretiden...","[Following, the, death, of, District, Attorney...",2012,"[ChristianBale, MichaelCaine, GaryOldman, Anne...",[ChristopherNolan]
4,49529,john carter,"[Action, Adventure, ScienceFiction]","[basedonnovel, mars, medallion, spacetravel, p...","[John, Carter, is, a, war-weary,, former, mili...",2012,"[TaylorKitsch, LynnCollins, SamanthaMorton, Wi...",[AndrewStanton]
...,...,...,...,...,...,...,...,...
4800,9367,el mariachi,"[Action, Crime, Thriller]","[unitedstates–mexicobarrier, legs, arms, paper...","[El, Mariachi, just, wants, to, play, his, gui...",1992,"[CarlosGallardo, JaimedeHoyos, PeterMarquardt,...",[RobertRodriguez]
4801,72766,newlyweds,"[Comedy, Romance]","[U, n, k, n, o, w, n]","[A, newlywed, couple's, honeymoon, is, upended...",2011,"[EdwardBurns, KerryBishé, MarshaDietlein, Cait...",[EdwardBurns]
4802,231617,"signed, sealed, delivered","[Comedy, Drama, Romance, TVMovie]","[date, loveatfirstsight, narration, investigat...","[""Signed,, Sealed,, Delivered"", introduces, a,...",2013,"[EricMabius, KristinBooth, CrystalLowe, GeoffG...",[ScottSmith]
4803,126186,shanghai calling,"[U, n, k, n, o, w, n]","[U, n, k, n, o, w, n]","[When, ambitious, New, York, attorney, Sam, is...",2012,"[DanielHenney, ElizaCoupe, BillPaxton, AlanRuck]",[DanielHsia]


In [163]:
df10.to_csv('movies.csv', index = False)

In [73]:
movies_length = [sum(len(j) for j in i) for i in df10['keywords']]
df10['keywords'].loc[1]

['ocean',
 'drugabuse',
 'exoticisland',
 'eastindiatradingcompany',
 "loveofone'slife",
 'traitor',
 'shipwreck',
 'strongwoman',
 'ship',
 'alliance',
 'calypso',
 'afterlife',
 'fighter',
 'pirate',
 'swashbuckler',
 'aftercreditsstinger']

In [71]:
import pickle

In [73]:
pickle.dump(df9.to_dict(), open('movies.pkl', 'wb'))

In [75]:
pickle.dump(similarity, open('similarity.pkl', 'wb'))